# Modeling and Predicting Student Dropout in Yiya Air Science Courses

In this project, our first aim to identify the points in the Yiya Air Science courses 2 and 3 where students dropout. There are different ways extents and ways to measure drop out. For example, dropout can be defined as a:
1. **Usage Gap**, where the time since the students last response is more than a certain amount of days e.g (e.g more than 7 days since last response). This can be seen as students being "on track" or "behind". We'll know if a user has performed a usage gap if the time between their most recent and last response to the system is more than N days. A reasonable value for N would be 7 days because the course modules are delivered weekly.
    - Alternatively, a usage gap can be defined as a user not sending any message to the system within a certain week.
    - Usage Gap = 1, if R2 - R1 > 7 days AND user_tasks < tasks_released, where R2= time of response 2, R1 = time of response 1
    - A response is any logged entry to the response table.
    
    `if (date_of_response_2 - date_of_response_1) > 7_days AND user_completed_tasks < tasks_released:`
        
        usage_gap == 1


2. **Total Question Stop**, where a student stops submitting scored_problems in the course before completing all tasks [(Borella, et al., 2019)](https://mitili.mit.edu/sites/default/files/project-documents/a24-Borrella_Caballero_Ponce_2019.pdf)
    - Total Question Stop = 1, if R2 - R1 > 7 days AND user_scored_problems < 50%_of_scored_problems_released, where R2= time of response 2, R1 = time of response 1
    - ❗️**Input Needed:** Is there an overall schedule of when tasks are accessible to users?
4. **Total Completion Stop**, where a student stops completing tasks (lesson modules and tests)
    - Tasks can be lessons, tests, or surveys
    
5. **Opt out** - student voluntarily unenrolls from the course without achieving a certificate [(Tinto, 1975)](https://www-tandfonline-com.cmu.idm.oclc.org/doi/full/10.1080/01587919.2017.1369006#)

Our second aim is to measure and visualize the rate of each of these dropout metrics among students over time and each course step. We can do so by diving the total occurances by the number of studnets. (Counts of students meeting dropout criteria / total number of students).

Our third aim is to identify features (variables) that are predictive of dropout with a classification model. Prior research suggests a need to refine success and dropout metrics [(Kohler, 2019)](https://www-tandfonline-com.cmu.idm.oclc.org/doi/full/10.1080/01587919.2017.1369006). It is uncertain if we have all of the following features in our data. However, we can try features grounded in previous theories, empircal results, and our best judgement on what makes sense for the context of the learning environment and the student population.  but some features that we may be able to use in our model and evaluate for predictive power include: 
- Student's date of first submission of a problem [(Liyanagunawardena, 2014)](https://centaur.reading.ac.uk/36002/)
- Student Intentions [(Liyanagunawardena, 2014)](https://centaur.reading.ac.uk/36002/)
    - Unsure
    - Browse Course
    - Audit
    - Complete
- Participation type [(Kohler, 2013)](https://er.educause.edu/articles/2013/6/retention-and-intention-in-massive-open-online-courses-in-depth)
    - Browser
    - Passive Participant 
    - Active Participant
    - Community Contributor
We can try to align participant types with [Yiya User Definitions](https://docs.google.com/document/d/1KNgPFdRSBniQouaKIQQPuI-DlAL-2xs70t4rpps5Qq4/edit)
- Completion Behavior
    - Number of lesson modules (tasks) completed
    - Number of lesson questions completed
    - Number of questions answered correctly
    - Number of tests they've completed 
    - Whether they are a returning learner - There should be a record for each Registration, Previously registered key in response as well. Data team will have to get back to us on how to best query the data to get an accurate user.
- Time on Task metric
    - Minutes spent on a task / Average time of session - We can use the telcomsession table and the created and duration feature. There can be telecom issues so users can get kicked out before completing the script. We'll have to consider this. We may have to compare with the tasks table.
- Review Behaviors 
    - Number of Times Revising Previous Lesson
- Effect of payment
    - Payment Status (Scholarship or Self Paid with mobile money) 
    - payment survey question - Required question in the response table. Shariffa will share the key. 
    
Note about "effect of payment": The date at which non-paid users lost access to course 3 was [*To be confirmed by yiya team, date was potentially nov 4th*]
Some required questions have required in their key?

# Exploring the Database

Yiya has a large database containing course content, user information, and interactions. The data spans multiple courses. The Entity Relationship Diagram below shows the relationships between the tables and the fields each table contains. We have the following tables:
- `content` - which describes the text or audio content users see or hear on their phones
- `outbound` - which tracks cellular data being from the system to users
- `inbound` - which tracks cellular data being to the system from users
- `response` - which describes the values users select from menus, surveys, lessons, tests, and other activities
- `user` - which describes the status of each user
- `task` - which records completions of tasks i.e completion of surveys, lessons, tests, etc.
- `registration` - which records each record of a user registration for a course
- `course` - which records each instance of a course run
- `channel`- **TECH TEAM INPUT NEEDED:** *the purpose of this table is unclear to the author of this notebook at time of writing*
- `telecommession` - which describes each session started on USSD by users.

❗️ Additional notes are needed on conditions under which each record is generated. 

**Recommendations for Future Tables and Fields**
- Survey table - Adding a table for surveys would help with the duplicate response problem in the response table. I recommend logging survey responses after the last survey question has been completed to ensure accurate survey data.
- Task Table Fields
    - time_taken - record the time between a user starting a tasks and completing a task after the task has been completed
    - score - record the score for lessons or tests after they have been completed. non scored tasks can be left empty.

Now that we have had an overview of the tables, we can quickly look at each table to identify data that can be used to model student dropout behaviors.

![Image of ERD][1]

[1]: /Users/ddbutler/repos_new/yiya_data_analysis/Documentation/ERD_2022-10-25.png

## Content Table
First, we'll examine the content table.

In [70]:
import pandas as pd

pd.set_option('display.max_columns', 1000) #show columns in scrollable table
pd.set_option('display.max_rows', 500)
pd.set_option("max_colwidth", None) #don't truncate data in columns.
pd.set_option('display.max_columns', 1000) #show columns in scrollable table
pd.set_option('display.max_rows', 500)
pd.set_option("max_colwidth", None) #don't truncate data in columns.

#read file from data folder, return file
def read_data(file_name, folder="/Users/ddbutler/Desktop/Repos/Yiya-Solutions-Analysis/yiya-completion-analysis/course3_data_v2_pickle/"):
    #combine foler and file name to get the full path
    df = pd.read_pickle(folder + file_name)
    return df

content_df = read_data(file_name="content.pkl")
content_df.sample(3, random_state=5) #See sample of data

,created,updated,script,section,version,kind,content,correct_value
id,,,,,,,,
22962,2022-09-29 18:42:11,2022-09-29 18:42:11,content/overview-yiya-airscience-course-2021-overview,.,zE2zM35idcz-xF38YMsAWQ,text,"Intro: What is STEM?\n1: Identify the problem\n2: Investigate\n3: Brainstorm\n4: Plan\n5: Create\n6: Test\n7: Improve\n8: Launch\n0. moto sandra, Continue...",None
38423,2022-10-18 10:38:10,2022-10-18 10:38:10,content/airscience-2022a/course/step-3-brainstorm/step-3-lesson-1/go-to-questions,q1-which-technology-are-we-going-to-create-in-this-course,p1i5dVVWfi1ZAPsvqpZDYg,text,That's not it OLIK JOSEPH... Try again.\n1/4: Which technology are we going to create in this course?\n1. Pedal powered washing machine\n2. An electric washing machine,Pedal powered washing machine
30187,2022-10-07 10:27:15,2022-10-07 10:27:15,content/airscience-2022a/course/step-1-identify/step-1-lesson-4/go-to-questions,what-is-6-out-of-10-as-a-percentage,cBR0we9JPtr2JmCRlp2ELw,text,Thank you P for answering lesson questions!,None


Based on this sample of the data we'll assume the following descriptions of the most relevant fields: 
- `created`: The created field is most likely the time content was served to the user
- `script` : The script field seems to be the activity being presented to a user.  *TODO❗️* There's a pattern that need to be described e,g
    - `content/airscience-2022a/course/step-3-brainstorm/step-3-lesson-1/go-to-questions`
    - `content/airscience-2022a/course/step-1-identify/step-1-lesson-4/go-to-questions`
    - `content/overview-yiya-airscience-course-2021-overview`
- `section`: The section field seems to describe the specific question or action within a script activity. 
- `version`: **Tech Team Input Needed**
- `kind`: The kind field suggests there are different types of content. *TODO❗️* We will look at the types of content later.
- `content`: The content field seems to be what the user actually sees or hears. 
- `correct_value`: The correct values field seems to mark the correct value for lesson and tests.

Now that we have an understanding of the field descriptions, we can look at the contents of the content data table in bit more more detail.


In [71]:
#content_df.describe(datetime_is_numeric=True, include="all")
#Input: a dataframe
#Output: dataframe describing data
#Description: generates summary of information about dataframe
def explore_data(df):
    #Set display options for easier viewing
    pd.set_option('display.max_columns', 1000) #show columns in scrollable table
    pd.set_option('display.max_rows', 500)
    pd.set_option("max_colwidth", None) #don't truncate data in columns.

    #See information on fields
    print("Information on Fields")
    print("----------------------")
    df.info()

    #See info on missing data
    print("\nPercentage of Missing Values")
    print("----------------------")
    print((df.isna().sum() / df.shape[0]) * 100)

    return df.describe(datetime_is_numeric=True, include="all")

#Check kinds of content
print("Kinds of Content: ",content_df["kind"].unique())
print("---")
#Print data summary
explore_data(content_df)

Kinds of Content:  ['text' 'audio']
---
Information on Fields
----------------------
<class 'pandas.core.frame.DataFrame'>
Int64Index: 78532 entries, 1 to 78562
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   created        78532 non-null  datetime64[ns]
 1   updated        78532 non-null  datetime64[ns]
 2   script         78531 non-null  object        
 3   section        78531 non-null  object        
 4   version        78461 non-null  object        
 5   kind           78532 non-null  object        
 6   content        78532 non-null  object        
 7   correct_value  73517 non-null  object        
dtypes: datetime64[ns](2), object(6)
memory usage: 5.4+ MB

Percentage of Missing Values
----------------------
created          0.000000
updated          0.000000
script           0.001273
section          0.001273
version          0.090409
kind             0.000000
content          0.000000
cor

,created,updated,script,section,version,kind,content,correct_value
count,78532,78532,78531,78531,78461,78532,78532,73517
unique,NaN,NaN,326,474,1067,2,62808,104
top,NaN,NaN,content/course-menu,.,cBR0we9JPtr2JmCRlp2ELw,text,Welcome to Yiya AirScience!\n1. On the Radio today\n2. Course Overview\n3. Revise previous content\n4. What to do next\n5. On the Radio today\n6. Course Overview\n7. What to do next Previous Lessons\n8. On the Radio today\n9. Course Overview\n10. Revise previous c,None
freq,NaN,NaN,6091,31981,6007,78398,632,52070
mean,2022-10-08 08:24:13.047101696,2022-10-08 08:24:13.047114496,NaN,NaN,NaN,NaN,NaN,NaN
min,2021-06-17 19:10:00,2021-06-17 19:10:00,NaN,NaN,NaN,NaN,NaN,NaN
25%,2022-09-26 14:11:26.750000128,2022-09-26 14:11:26.750000128,NaN,NaN,NaN,NaN,NaN,NaN
50%,2022-10-19 10:09:05.500000,2022-10-19 10:09:05.500000,NaN,NaN,NaN,NaN,NaN,NaN
75%,2022-11-12 21:29:16.249999872,2022-11-12 21:29:16.249999872,NaN,NaN,NaN,NaN,NaN,NaN
max,2022-12-28 13:20:07,2022-12-28 13:20:07,NaN,NaN,NaN,NaN,NaN,NaN


Look at the description of the data above, 78,532 instances of users receiving content. The data is fairly complete. The only column with significant missing values is the `correct_value` column, which suggests that about 6% of content served is non lesson or test. The dates range from `2021-06-17` to `2022-10-19`.

There are 326 unique scripts (activities) between the first to most recent course.  We have 2 kinds of content: `text` and `audio`.  The most recent content suggests the course menu, which serves as the entry point for users.

### Analysis Questions

Understanding the course table, allows us to ask some questions about dropout.
- Do students who complete the course listen to audio content at a higher rate?
- What content is communicated via audio?

In [72]:
audio_content = content_df.query("kind == 'audio'")
print("\nAudio Content Summary")
print("Unique Scripts", audio_content["script"].unique())
print("\nUnique Sections\n", audio_content["section"].unique())
audio_content.sample(3, random_state=5)



Audio Content Summary
Unique Scripts ['content/for-testers' 'content/for-registered' 'content/active_users'
 'content/airscience-2022a/schedules/for-testers'
 'content/airscience-2022b/schedules/for-testers'
 'content/airscience-2022a/schedules/for-registered'
 'content/airscience-2022b/schedules/for-registered']

Unique Sections
 ['Friday' 'Friday, June 25' 'Monday, June 28' 'Wednesday, June 30'
 'July 1' 'Wednesday July 07, 2021' 'Thursday July 08, 2021'
 'Friday July 09, 2021' 'Sunday July 11, 2021' 'Monday July 12, 2021'
 'Monday July 13, 2021' 'Sunday July 18, 2021' 'Sunday July 18'
 'Saturday July 24, 2021' 'Sunday July 25, 2021' 'Sunday July 25'
 'Thursday July 29' 'Sunday August 1, 2021' 'Monday August 09, 2021'
 'Tuesday August 10, 2021' 'Tuesday August 10' 'Sunday August 15,2021'
 'Friday  August 20, 2021' 'Friday August 27, 2021'
 'Friday September 17, 2021' 'Sunday, September 19'
 'Friday October 1, 2021' 'Sunday, October 3' 'Monday, October 4'
 'Tuesday, October 5' 'Thurs

,created,updated,script,section,version,kind,content,correct_value
id,,,,,,,,
1141,2021-07-24 07:00:01,2021-07-24 07:00:01,content/for-testers,"Saturday July 24, 2021",o-YlVlmqLRwtXmEmthIjIg,audio,../.gitbook/assets/step_2_robocall.mp3,NaN
1483,2021-08-10 13:00:07,2021-08-10 13:00:07,content/for-registered,Tuesday August 10,NaN,audio,../.gitbook/assets/step3_brainstorm_robocall.mp3,NaN
47071,2022-10-30 13:00:04,2022-10-30 13:00:04,content/airscience-2022a/schedules/for-registered,"Sunday, October 30, 2022",cO-o8BWsu1Q-5xoPrDfeFQ,audio,../../../.gitbook/assets/Course_3_Step_5_Robocall.mp3,NaN


Looking at the sample, the `created `dates tell us that audio content is used for multiple courses and the sections are just labels for the days the content was sent out. 

#### **Do students who persist listen to audio content at a higher rate?** - 
It looks like the content table can't answer this question because there's no easy link with user_id. However, the `channel table` has fields for `user_id` and `kind`. We'll mine that data from there later. Although, the content description for audio content does have useful labels for names that suggest which part of the course the audio is for. What content is communicated via audio?
- **TODO:** Explore the channel table to investigate what kinds of content groups of learners are requesting.

#### **What content is communicated via audio?** 
Based on the `content` field, These seem to be mostly robocalls. Based on the curriculum documentation, robocalls are previews of what is coming up in the course. 
- **TODO❗️**: A Question for yiya team, what is the purpose of robocalls? When are they sent out in respect to broadcasts? Can robocalls be requested by users any time?

## Outbound Table
Now that we've explored the content table, let's look at the outbound table.


In [73]:
outbound_df = read_data("outbound.pkl")
outbound_df.sample(3, random_state=6)


,created,updated,channel_id,status,content_id,header
id,,,,,,
659838,2021-07-16 11:49:16,2021-07-16 11:49:16,27340,delivered,308,NaN
3259076,2022-09-18 17:52:28,2022-09-18 17:52:28,82142,delivered,5398,NaN
2339242,2021-10-01 10:55:07,2021-10-01 10:55:07,19165,delivered,2028,NaN


Based on the sample, we can describe the potentially important fields as follows:
- `created` = time message was sent out
- `channel_id` = foreign key linking to channel table
- `status` = whether message was delivered
- `content_id` = foreign key link to content table
-  `header` = cannot be understood from sample

The status and header field need further exploration.

In [74]:
unique_status = outbound_df["status"].unique()
print("Unique Status\n", unique_status)

unique_headers = outbound_df["header"].unique()
print("Unique Headers Sample\n", unique_headers[0:5])

explore_data(outbound_df)

Unique Status
 ['delivered']
Unique Headers Sample
 [nan
 'READ_MORE_SMS - content/course-spring-2021/intro-step/intro-step-lesson-3'
 'READ_MORE_SMS - content/solar-food-dryer'
 'READ_MORE_SMS - content/course-spring-2021/step-5-create/step-5-lesson-4'
 'READ_MORE_SMS - content/solar-food-dryer/create-solar-food-dryer']
Information on Fields
----------------------
<class 'pandas.core.frame.DataFrame'>
Int64Index: 4092930 entries, 1 to 4095388
Data columns (total 6 columns):
 #   Column      Dtype         
---  ------      -----         
 0   created     datetime64[ns]
 1   updated     datetime64[ns]
 2   channel_id  int64         
 3   status      object        
 4   content_id  int64         
 5   header      object        
dtypes: datetime64[ns](2), int64(2), object(2)
memory usage: 218.6+ MB

Percentage of Missing Values
----------------------
created        0.000000
updated        0.000000
channel_id     0.000000
status         0.000000
content_id     0.000000
header        99.733

,created,updated,channel_id,status,content_id,header
count,4092930,4092930,4.092930e+06,4092930,4.092930e+06,10890
unique,NaN,NaN,NaN,1,NaN,80
top,NaN,NaN,NaN,delivered,NaN,AUTO_TEST_REPORT - content/yiya-airscience-tests-2021/intro-step-test
freq,NaN,NaN,NaN,4092930,NaN,3022
mean,2021-12-09 21:19:34.527879168,2021-12-09 21:19:34.527888384,9.627785e+04,NaN,8.816809e+03,NaN
min,2021-06-17 19:10:01,2021-06-17 19:10:01,1.000000e+00,NaN,1.000000e+00,NaN
25%,2021-07-28 06:13:34,2021-07-28 06:13:34,5.192700e+04,NaN,2.610000e+02,NaN
50%,2021-09-14 17:06:51.500000,2021-09-14 17:06:51.500000,1.002750e+05,NaN,1.586000e+03,NaN
75%,2022-03-23 13:04:07,2022-03-23 13:04:07,1.310700e+05,NaN,2.319000e+03,NaN
max,2022-12-28 13:55:05,2022-12-28 13:55:05,2.314450e+05,NaN,7.856200e+04,NaN


There are `4,092,930` entries in the outbound table. The only value for the `status` field is `delivered`, which implies that failed messages are not saved to the database. Also, the `header` field seems to have null values or be a conbination of the stirng `READ_MORE_SMS -` and a `script` like `content/course-spring-2021/step-5-create/step-5-lesson-4'`. The outbound table alone seems unimportant for analysis beyond merging other tables given this interpretation.

## Inbound Table

Let's look at the inbound table. There may not be a much to leverage, but a quick look may be useful. 

In [79]:
inbound_df = read_data("inbound.pkl")
inbound_df.sample(3, random_state=6)

,created,updated,channel_id,prompt_id,text
id,,,,,
1371308,2021-08-27 19:03:45,2021-08-27 19:03:45,130046,1782895.0,5feceb66ffc86f38d952786c6d696c79c2dbc239dd4e91b46729d73a27fb57e9
1819876,2021-10-07 16:28:49,2021-10-07 16:28:49,123928,2459926.0,6b86b273ff34fce19d6b804eff5a3f5747ada4eaa22f1d49c01e52ddb7875b4b
713766,2021-07-21 18:21:25,2021-07-21 18:21:25,20538,884964.0,6b86b273ff34fce19d6b804eff5a3f5747ada4eaa22f1d49c01e52ddb7875b4b


Based on the sample above, the inbound table alone does not have anything interesting beyond the ability the link the other tables. The fields have include:
- created =
- channel_id - foreign key to link channel table
- prompt_id - foreign key to link response table

## Channel Table
**TODO:** Identify the purpose of the channel table. `channel`- **TECH TEAM INPUT NEEDED:** *the purpose of this table is unclear to the author of this notebook at time of writing*

Let's explore the channel table.

In [75]:
channel_df = read_data("channel.pkl")
channel_df.sample(3, random_state=6)

,created,updated,user_id,kind,enabled,phone,state,task
id,,,,,,,,
10590,2021-06-22 03:18:12,2021-06-22 03:18:12,10267,ussd,1,256770557674,NaN,NaN
150862,2021-11-28 05:48:35,2021-11-28 05:48:35,50858,sms,1,256783223267,NaN,NaN
74687,2021-07-02 09:14:50,2021-07-02 09:14:50,23559,sms,1,256779927525,NaN,NaN


We see the following fields
- created, which is time record was created in YY-MM-DD format
- user_id, which is id of user
- kind, which describes the 
- enabled, which seems to be a true of false value. **TODO:** Investigate the un
- phone - phone number of user contacted? **TODO** Verify this description with tech team
- state - **TODO:** Verify the description of this field with tech team.
- task - **TODO:** Verify the description of this field with tech team.

There is a task table and a task field in this database, but tasks seems to be mostly null based on the sample. Let's explore the `task`, `state`, and `kind` fields in more detail.

In [76]:
#Input: a numpy array of unique values
#Output: a numpy array
def sample_array(the_array, size=5, seed=1):
    # https://numpy.org/doc/stable/reference/random/generated/numpy.random.Generator.choice.html#numpy.random.Generator.choice
    import numpy as np
    rng = np.random.default_rng(1) #Call default_rng (random generator) to get a new instance of a Generator with a seed
    random_sample = rng.choice(the_array, size)
    return random_sample

#Print sample of unique channel states
unique_state = channel_df["state"].unique()
state_sample = sample_array(unique_state, size=5, seed=1)
print("Unique Channel States\n", state_sample,)
print("Num of Unique: ", channel_df["state"].nunique(), "\n")

#print unique tasks 
unique_tasks = channel_df["task"].unique()
task_sample = sample_array(unique_tasks, size=5, seed=2)
print("Unique Task Sample\n", task_sample)
print("Num of Unique: ", channel_df["task"].nunique(), "\n")

explore_data(channel_df)

Unique Channel States
 ['content/course-menu/~zZIJ0NBhHOqBA7sIplCIhg'
 'content/course-spring-2021/step-2-investigate/step-2-lesson-3/take-quiz/~AqoczGE1_tT5aTIMnH4QGw'
 'content/onboarding/registration-selection/~'
 'content/onboarding/registration-by-student/~Chb8ucZ0jXyh0vqN0UHa5A/:/content/profile/gender/~uw_Zbsob-oWlDa50fo6JWw'
 'content/course-spring-2021/assessment-questions/~']
Num of Unique:  906 

Unique Task Sample
 ['content/course-spring-2021/step-6-test/step-6-lesson-1'
 'content/yiya-airscience-tests-2021/take-step-3-test'
 'content/airscience-2022a/course/tests/step-4-test'
 'content/airscience-2022a/course/step-7-improve/step-7-lesson-1'
 'content/solar-food-dryer']
Num of Unique:  113 

Information on Fields
----------------------
<class 'pandas.core.frame.DataFrame'>
Int64Index: 232316 entries, 1 to 235181
Data columns (total 8 columns):
 #   Column   Non-Null Count   Dtype         
---  ------   --------------   -----         
 0   created  232316 non-null  datetime

,created,updated,user_id,kind,enabled,phone,state,task
count,232316,232316,232316.000000,232316,232316.0,2.323160e+05,38408,34827
unique,NaN,NaN,NaN,4,NaN,NaN,906,113
top,NaN,NaN,NaN,ussd,NaN,NaN,content/onboarding/~8F2BRCvsYUW7siifjAItNw,content/onboarding
freq,NaN,NaN,NaN,77161,NaN,NaN,6856,25666
mean,2021-12-12 22:17:37.065053440,2021-12-23 17:25:38.536377600,39527.478749,NaN,1.0,5.036789e+11,NaN,NaN
min,2021-06-17 15:53:28,2021-06-17 15:53:28,1.000000,NaN,1.0,7.600088e+08,NaN,NaN
25%,2021-07-02 09:13:12,2021-07-02 09:14:09,20072.000000,NaN,1.0,2.567728e+11,NaN,NaN
50%,2021-07-29 05:09:21,2021-08-19 17:58:44,39569.000000,NaN,1.0,2.567792e+11,NaN,NaN
75%,2022-09-14 14:50:36,2022-09-15 17:18:17.249999872,58809.000000,NaN,1.0,2.567861e+11,NaN,NaN
max,2022-12-28 14:07:12,2022-12-28 14:07:12,78691.000000,NaN,1.0,2.567900e+12,NaN,NaN


The `state` field has directory strings like `'content/course-spring-2021/step-2-investigate/step-2-lesson-3/take-quiz/~AqoczGE1_tT5aTIMnH4QGw'`. A significant difference is the string of characters like `~AqoczGE1_tT5aTIMnH4QGw` that follows at the end. About 83% of entries in the chanel table are missing values in the state column. 

The `task` field has endpoints for 2022 and 2021 e.g `'content/airscience-2022a/course/tests/step-4-test'`.
***TODO:*** Ask the following questions to data team:
1. Ask the data team what the difference between these state, tasks in channel and script in other tables.
2. Why are 83% and 85% of state and task table missing data?

### Analysis Questions
- What kind of channels are used most?
- Do student who listen to audio at a higher rate persist better? - Data needs to be combined to answer this question.

In [77]:
def get_count_report(df, column):
    counts_norm = df[column].value_counts(normalize=True) * 100 #get normalized values and multply by 100 for percentage
    counts = df[column].value_counts() #raw counts

    report = pd.merge(counts_norm, counts, right_index = True,
                left_index = True, suffixes=("_perc", "_count")) #merge into dataframe
    return report
ch_kind_counts = get_count_report(channel_df, "kind")
ch_kind_counts

,kind_perc,kind_count
ussd,33.213812,77161
sms,33.213812,77161
voice,33.213812,77161
simulator,0.358563,833


#### What kind of channels are used most?
From the above questions, it looks like ussd, sms, and voice were used at the same rates, ~33%.
- ***TODO**: Why were these channels used at the same rates?

# User Table
Let's explore the user table.

In [87]:
user_df = read_data("user.pkl")
user_df.sample(3, random_state=10)

,created,updated,phone,tester,active,opt_out
id,,,,,,
45717,2021-09-17 10:59:48,2021-09-17 10:59:48,96d1e72e082d221b89096b018d5c824cd01433c1aa22ec58a1fec742bcb2c83a,0,0,NaN
15608,2020-09-30 09:47:38,2021-06-22 03:29:50,57e500cc992f03c8b8d994de891bc7065d11f4eee5a09b4c32eed5f49978032f,0,0,NaN
65561,2022-09-23 11:37:38,2022-09-23 11:37:38,bdeb1f068d1a6eead6a68fd0853ba1740174df87efb0ba1d34cb096c7ea6175c,0,0,NaN


The columns we have are
- `created` DATETIME  = the date and time user record was added
- `updated` DATETIME = **TODO:** Do they updated fields matter anywhere?
- `phone` VARCHAR(255) = hashed phone number ***TODO:*** Ask, Do the same phone numbers get hashed to the same value?
- `type` TINYINT(1) = according to the ERD diagram there is a type field, but it does not exist here. **TODO:** Why?
- `tester` TINYINT(1) = binary flag to indicate whether a user account is a tester or not. ***TODO:*** Where do testers comes from? Are they created in the simulator?
- `active` TINYINT(1) 
- `opt_out` DATETIME = date when a user selected to stop receiving messages.

Let's explore these fields in more depths.